In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt 
import numpy as np
from functools import partial
import json
from glob import glob

import cv2
import os
import random

import csv
import pandas as pd
import shutil

from scipy import interpolate
#import nia22
#from nia22.generator import *

In [3]:
def get_converter(field:str):
    if field == "inst":
        keys = ["S", "T", "L", "V", "M"]
    elif field == "condition":
        keys = ["F", "S", "D", "A", "N"]
    elif field == "posture":
        keys = ["S", "D", "P", "L", "F", 
                "C", "H", "E", "T", "U"]
    else:
        raise ValueError(field)

    return partial(_concat, keys=keys)

def _concat(value, keys=None):
    """concatenate strings"""
    code =""
    for s in keys:
        if s in value:
            code += s
        else:
            code += "X"
    return code

# get total file size
def get_total_file_size(wdir, pattern, flist=False):
    all_files = glob(wdir+ pattern)
    all_files.sort()
    nfiles = len(all_files)
    
    print(f"found {nfiles} files")
    
    file_sizes_Bytes = [os.path.getsize(cc) for cc in all_files]
    tsize_inMB = np.sum(file_sizes_Bytes) / 1024 / 1024
    
    all_file_names = [al.split("processing/")[1] for al in all_files]
    if tsize_inMB > 1024:
        if flist:
            return nfiles, f"{tsize_inMB/1024:.2f}GB", all_file_names
        else:
            return nfiles, f"{tsize_inMB/1024:.2f}GB"
    else:
        if flist:
            return nfiles, f"{tsize_inMB:.2f}MB", all_file_names
        else:
            return nfiles, f"{tsize_inMB:.2f}MB"

def change_age(age):
    if age.startswith("1"):
        age = 10
    elif age.startswith("2"):
        age = 20
    elif age.startswith("3"):
        age = 30
    elif age.startswith("4"):
        age = 40
    elif age.startswith("5"):
        age = 50
    elif age.startswith("6"):
        age = 60
    return age

In [25]:
import pandas as pd
import csv, json
from datetime import datetime

import numpy as np

xlsx = pd.read_excel('/home/di/Work/nia/메타데이터.xlsx')
dates = [f"{dd.month:02d}{dd.day:02d}" for dd in xlsx['촬영날짜']]
xlsx = list(xlsx.values)

In [6]:
concat_inst = get_converter("inst")
concat_condition = get_converter("condition")
concat_posture = get_converter("posture")

In [30]:
nas_dir = ['/media/di/mx500_2tb2/NIA2022_2/', 
           "/media/di/data/NIA2022/",
           "/run/user/1000/gvfs/sftp:host=192.168.0.43,port=5002/NIA2022/raw/"][2]

### 공통
loc = "S101"
inst = "STLVM"
condition = "FSDAN"
posture = "SDPLFCHETU"
###
for data in xlsx[35:]:
    clip_list_fn =[]
    
    name, gender, subject, medical, age, glasses, mask, cosmetic, ver, date = data[:10]
    subject = int(subject)
    date, hour = str(date).split(" ")
    hour = hour[:-3]
    
    ## Variables below are to be read from csv file ##
    username = "hoseung"
    email = "hschoi@dinsight.ai"
    glasses = "no" if glasses == "미착용" else "yes"
    cosmetic= "no"
    mask = "no" if mask == "미착용" else "yes"
    gender = "male" if gender == "남" else "female"
    
    rev = str(0)

    wdir = nas_dir + f'{"".join(date.split("-")[1:])}/processing/S1/{subject:03d}/T1/*/'
    
    n_point, tsize_point = get_total_file_size(wdir+"Eye-tracker/", "*.csv")
    n_head, tsize_head = get_total_file_size(wdir+"FaceAngle/", "*.csv")
    n_cam, tsize_cam = get_total_file_size(wdir+"CamAngle/", "*.csv") 
    n_dcam, tsize_d_cam = get_total_file_size(wdir+"DistCam2Face/", "*.csv") # was display pose
    n_disp, tsize_d_disp = get_total_file_size(wdir+"DistDisp2Face/", "*.csv")
    n_rgb, tsize_rgb, cl_rgb = get_total_file_size(wdir+"RGB/", "*_rgb_*.mp4", flist=True)
    n_ir, tsize_ir, cl_ir = get_total_file_size(wdir+"IR/", "*_ir_*.mp4", flist=True)
    clip_list_fn = cl_rgb + cl_ir
    ######################################
    
    ver  = n_cam
    
    x = {
        "id": {"subject":subject, 
               "medical":medical},
        "age":change_age(age), 
        "glasses":glasses, 
        "cosmetic":cosmetic, 
        "mask":mask, 
        "gender":gender,
        "ver":ver, 
        "date":date, 
        "loc":loc, 
        "inst":concat_inst(inst), 
        "condition":concat_condition(condition),
        "posture":concat_posture(posture),
        "rev":rev,
        "image": {"format": {"rgb": "mp4","ir": "mp4", "point":"csv", "head":"csv", 
                    #         "gaze":"csv", 
                             "cam":"csv",
                             "d_cam":"csv", "d_disp":"csv"},
                    "spec":{"rgb.w":1920, "rgb.h":1080, "ir.w":1920, "ir.h":1080, 
                             "point":"x,y",
                             "head":"deg,deg,deg",
                    #         "gaze":"yyyy-mm-dd,hh:mm:ss,deg,deg,deg",
                             "cam":"deg,deg,deg",
                             "d_cam":"cm",
                             "d_disp":"cm"},
                    "total_size":{"rgb":tsize_rgb, "ir":tsize_ir, "point":tsize_point, "head":tsize_head,
                    #              "gaze":tsize_gaze, 
                                  "cam":tsize_cam, "d_cam":tsize_d_cam, "d_disp":tsize_d_disp},
                    "total_count":{"rgb":n_rgb, "ir":n_ir, "point":n_point, "head":n_head,
                    #               "gaze":n_gaze,
                                   "cam":n_cam, "d_cam":n_dcam, "d_disp":n_disp}
                 },
        "task":{
            "name":"NIA_EYE",
            "created": date + ", " + hour, 
            "updated":"", 
            "username":username, 
            "email":email},
        #"filelist":clip_list_fn
    }

    # Dump json file
    fn = f"{subject:03d}.json"
    with open(fn, "w") as f:
        json.dump(x, f, indent=2, ensure_ascii=False)

found 0 files
found 453 files
found 453 files
found 453 files
found 453 files
found 0 files
found 0 files
found 0 files
found 299 files
found 452 files
found 452 files
found 452 files
found 0 files
found 0 files
found 0 files
found 453 files
found 453 files
found 453 files
found 453 files
found 0 files
found 0 files
found 0 files
found 102 files
found 453 files
found 453 files
found 453 files
found 0 files
found 0 files
found 0 files
found 0 files
found 452 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 453 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 0 files
found 453 files
found 0 files
found 0 files
found 0 files
found 0 files
found 453 files
found 453 files
found 453 files
found 453 files
found 453 files
found 453 files
found 126 files
found 453 files
found 453 files
found 453 files
found 453 files
found 453 files
found 453 files
found 126 files
found 453 files
found 453 files
found 450 files
foun

In [24]:
fn

'055.json'

In [7]:
def csv_to_arr(fn, skip_header=True):
    with open(fn, "r") as f:
        f.seek(0)
        if skip_header: f.readline()
        rrr = csv.reader(f, quoting=csv.QUOTE_NONNUMERIC)
        all_lines = []
        for rr in rrr:
            all_lines.append(rr)

    return np.array(all_lines).astype(float)

def matching_string(string, slist):
    for i, ss in enumerate(slist):
        if string in ss:
            return i

get_pose = lambda x : x.split("/")[-1][-9:]
get_fn   = lambda x : x.split("/")[-1]

In [72]:
date = 4
device=["Smartphone", "Tablet"][0]
wgood = f'/media/di/data/NIA2022/08{date:02d}/processing/S1/006/T1/'
wbad = f'/media/di/data/NIA2022/08{date:02d}/processing/S1/007/T1/'

fgood = glob(wgood + f"{device}/CamAngle/*.csv")

fbadtarget = glob(wbad + f"{device}/DistCam2Face/*.csv")
fbad  = glob(wbad  + f"{device}/CamAngle/*.csv")
fgood.sort()
fbad.sort()
fbadtarget.sort()

all_good_fn = [*map(get_fn, fgood)]
all_bad_fn =  [*map(get_fn, fbad)]
all_bad_target =  [*map(get_fn, fbadtarget)]

all_good_pose = [*map(get_pose, fgood)]
all_bad_pose =  [*map(get_pose, fbad)]
all_bad_target_pose =  [*map(get_pose, fbadtarget)]
all_common_pose = np.intersect1d(all_good_pose, all_bad_pose)

missing = []
for this_csv in all_bad_target:
    if this_csv.replace("_dcam_", "_cam_") not in all_bad_fn:
        missing.append(this_csv)

# missing = []
# for good in all_bad_target: # 같은 사람, 다른 기기여야함. 다른 사람이면 안 돼! 
#     if good not in all_bad_pose:
#         missing.append(good)

ind_arr = np.arange(len(all_common_pose))
for missing_fn in missing:
    good_missing = fgood[matching_string(missing_fn[-9:], fgood)]
    
    random.shuffle(ind_arr)
    
    ii = 0
    while True:
        # r,p,y 모두 90도 안에 들어올 때 까지 기준점을 랜덤으로 고름
        iorg = ind_arr[ii]
        comm_key = all_common_pose[iorg]
        good_origin = fgood[matching_string(comm_key, fgood)]
        bad_origin  = fbad[matching_string(comm_key, fbad)]
        
        arr0 = csv_to_arr(good_origin)
        arr_g = csv_to_arr(good_missing)
        arr0b = csv_to_arr(bad_origin)

        arr_bad_missing = arr0b - (arr0 - arr_g)
        
        if np.max(np.abs(arr_bad_missing)) < 180:
            break
        else:
            ii+=1

    # Save to a new file
    fn_bad_missing = wbad  + f"{device}/CamAngle/" + missing_fn.replace("_dcam_", "_cam_")

    with open(fn_bad_missing, 'w') as f:
        f.write(f"Roll Pitch Yaw\n")
        for v1,v2,v3 in arr_bad_missing:
            f.write(f"{v1:.3f}, {v2:.3f}, {v3:.3f}\n")